In [3]:
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# 1. 데이터 로드 및 분할
print("[1] MNIST 데이터 불러오기")
mnist = fetch_openml('mnist_784', version=1, as_frame=False)
X, y = mnist.data.astype(np.float32), mnist.target.astype(np.int64)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=10000, stratify=y, random_state=42
)
print(f"훈련 데이터: {X_train.shape}, 테스트: {X_test.shape}")

# 2. 전처리 (정규화)
print("[2] 데이터 정규화")
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 3. KNN 모델 최적 파라미터 설정 및 평가
print("[3] KNN 모델 학습 및 하이퍼파라미터 탐색")
best_acc = -1.0
for n in [3, 4, 5]:
    for w in ['uniform', 'distance']:
        knn = KNeighborsClassifier(n_neighbors=n, weights=w, n_jobs=-1)
        knn.fit(X_train_scaled, y_train)
        preds = knn.predict(X_test_scaled)
        acc = accuracy_score(y_test, preds)
        print(f"  k={n}, weights='{w}' → acc={acc*100:.2f}%")
        if acc > best_acc:
            best_acc = acc
            best_knn = knn

# 4. 데이터 증강 함수
def shift_image(arr, dx, dy):
    img = arr.reshape(28,28)
    shifted = np.roll(img, shift=dy, axis=0)
    shifted = np.roll(shifted, shift=dx, axis=1)
    if dy > 0: shifted[:dy,:] = 0
    if dy < 0: shifted[dy:,:] = 0
    if dx > 0: shifted[:,:dx] = 0
    if dx < 0: shifted[:,dx:] = 0
    return shifted.flatten()

print("[4] 데이터 증강 (4방향)")
aug_X = []
aug_y = []
for xi, yi in zip(X_train, y_train):
    aug_X.append(xi)
    aug_y.append(yi)
    for dx, dy in [(-1,0), (1,0), (0,-1), (0,1)]:
        aug_X.append(shift_image(xi, dx, dy))
        aug_y.append(yi)
aug_X = np.array(aug_X, dtype=np.float32)
aug_y = np.array(aug_y, dtype=np.int64)

# 5. 증강 데이터 전처리 및 평가
print("[5] 증강 데이터 정규화 후 학습")
aug_X_scaled = scaler.transform(aug_X)
knn.fit(aug_X_scaled, aug_y)
pred_aug = knn.predict(X_test_scaled)
aug_acc = accuracy_score(y_test, pred_aug)

print("\n--- 최종 결과 ---")
print(f"원본 훈련 데이터 정확도: {best_acc*100:.2f}%")
print(f"증강 데이터 훈련 정확도: {aug_acc*100:.2f}%")
print(f"증강 효과 (차이):        {(aug_acc-best_acc)*100:.2f} pp")


[1] MNIST 데이터 불러오기
훈련 데이터: (60000, 784), 테스트: (10000, 784)
[2] 데이터 정규화
[3] KNN 모델 학습 및 하이퍼파라미터 탐색
  k=3, weights='uniform' → acc=97.36%
  k=3, weights='distance' → acc=97.52%
  k=4, weights='uniform' → acc=97.19%
  k=4, weights='distance' → acc=97.58%
  k=5, weights='uniform' → acc=97.17%
  k=5, weights='distance' → acc=97.23%
[4] 데이터 증강 (4방향)
[5] 증강 데이터 정규화 후 학습

--- 최종 결과 ---
원본 훈련 데이터 정확도: 97.58%
증강 데이터 훈련 정확도: 98.00%
증강 효과 (차이):        0.42 pp
